In [ ]:
import argparse
import os

import torch

from videollava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN, \
    DEFAULT_VIDEO_TOKEN
from videollava.conversation import conv_templates, SeparatorStyle
from videollava.model.builder import load_pretrained_model
from videollava.serve.utils import load_image, image_ext, video_ext
from videollava.utils import disable_torch_init
from videollava.mm_utils import process_images, tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria

from PIL import Image

import requests
from PIL import Image
from io import BytesIO
from transformers import TextStreamer
import json

In [ ]:
# Model
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

disable_torch_init()


model_base = None
model_path = "local_model/Video-LLaVA-7B"

model_name = get_model_name_from_path(model_path)

tokenizer, model, processor, context_len = load_pretrained_model(model_path, model_base, model_name,
                                                                     device="cuda:0", cache_dir=None)
image_processor, video_processor = processor['image'], processor['video']

conv_mode = "v1_my"


In [ ]:
def eval_single_task(task):
    conv = conv_templates[conv_mode].copy()
    tensor = []
    outs = []
    for subtask in task:
        special_token = []

        for image_path in subtask["image"]:
            image_path = os.path.join("/data2/dpl_data/OctoGibsonDataset/all_data_images", image_path)
            image = image_processor.preprocess(image_path, return_tensors='pt')['pixel_values'][0].to(model.device, dtype=torch.float16)
            special_token += [DEFAULT_IMAGE_TOKEN]           
            tensor.append(image)

        inp = subtask["conversations"][0]["value"]
        if getattr(model.config, "mm_use_im_start_end", False):
            inp = ''.join([DEFAULT_IM_START_TOKEN + i + DEFAULT_IM_END_TOKEN for i in special_token]) + '\n' + inp
        else:
            inp = ''.join(special_token) + '\n' + inp


        conv.append_message(conv.roles[0], inp)
        
        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()
        print(prompt)
        input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)
        stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
        keywords = [stop_str]
        stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
        streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
        
        with torch.inference_mode():
            output_ids = model.generate(
                    input_ids,
                    images=tensor,  # video as fake images
                    do_sample=True,
                    temperature=0.2,
                    max_new_tokens=512,
                    streamer=streamer,
                    use_cache=True,
                    stopping_criteria=[stopping_criteria])

        outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
        conv.messages[-1][-1] = outputs
        outs.append(outputs)
    return outs   

In [ ]:
def eval_single_subtask(subtask):
    conv = conv_templates[conv_mode].copy()
    tensor = []
    special_token = []

    for image_path in subtask["image"]:
        image_path = os.path.join("/data2/dpl_data/OctoGibsonDataset/all_data_images", image_path)
        image = image_processor.preprocess(image_path, return_tensors='pt')['pixel_values'][0].to(model.device, dtype=torch.float16)
        special_token += [DEFAULT_IMAGE_TOKEN]           
        tensor.append(image)

    inp = subtask["conversations"][0]["value"]
    if getattr(model.config, "mm_use_im_start_end", False):
        inp = ''.join([DEFAULT_IM_START_TOKEN + i + DEFAULT_IM_END_TOKEN for i in special_token]) + '\n' + inp
    else:
        inp = ''.join(special_token) + '\n' + inp


    conv.append_message(conv.roles[0], inp)
        
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()
    
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
        
    with torch.inference_mode():
            output_ids = model.generate(
                    input_ids,
                    images=tensor,  # video as fake images
                    do_sample=True,
                    temperature=0.2,
                    max_new_tokens=512,
                    streamer=streamer,
                    use_cache=True,
                    stopping_criteria=[stopping_criteria])

    outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
    conv.messages[-1][-1] = outputs
 
    return outputs   

In [ ]:
'''
with open('/data2/dpl_data/OctoGibsonDataset/eval_merge.json', 'r') as file1:
    data = json.load(file1)

outs = eval_single_task(data[5])
print(outs)
'''

In [ ]:
with open('/data2/dpl_data/Video-LLaVA/videollava/serve/results.json', 'r') as file2:
    data = json.load(file2)

for subtask in data:
    output = eval_single_subtask(subtask)
    subtask["conversations"].append({
                "from": "llava",
                "value": output})


In [ ]:
file_path = "/data2/dpl_data/Video-LLaVA/results.json"


# 将列表写入 JSON 文件，并指定缩进为4个空格
with open(file_path, "w") as file:
    json.dump(data, file, indent=4)